In [135]:
latinobarometro = pd.read_stata('/content/Latinobarometro_2020_Esp_Stata_v1_0.dta')
# Limpiar espacios y normalizar mayúsculas/minúsculas
latinobarometro['idenpa_clean'] = latinobarometro['idenpa'].str.strip().str.upper()

In [136]:
conversion = {
    "No muy satisfecho": 1,
    "Para nada satisfecho": 2,
    "Bastante satisfecho": 3,
    "Muy satisfecho": 4
}

# Reemplazar valores en la columna P1ST
latinobarometro["felicidad"] = latinobarometro["p1st"].map(conversion)

In [137]:
import numpy as np

# Diccionario de conversión explícita
conversion = {
    "derecha": 10,
    "izquierda": 0,
    "Ninguno": np.nan,
    "No contesta": np.nan,
    "No sabe": np.nan,
    97: np.nan,
    99: np.nan,
    "97": np.nan,
    "99": np.nan
}

def convertir(val):
    # Si el valor está en el diccionario, usarlo
    if val in conversion:
        return conversion[val]

    # Si es numérico y no es 97/99, devolverlo tal cual
    try:
        # intentar convertir a número
        num = float(val)
        # si es 97 o 99, poner NaN igual por seguridad
        if num in [97, 99]:
            return np.nan
        return num
    except:
        # cualquier string que no esté en el dict lo dejamos como está
        return val

latinobarometro["escala"] = latinobarometro["p18st"].apply(convertir)

In [138]:
import numpy as np
from scipy.stats import pearsonr

# Reemplazar strings '#NA' por NaN
latinobarometro['felicidad'] = latinobarometro['felicidad'].replace("#NA", np.nan)
latinobarometro['escala'] = latinobarometro['escala'].replace("#NA", np.nan)

# Filtrar filas válidas (sin NaN)
df = latinobarometro[['felicidad', 'escala']].dropna()

# Calcular correlación de Pearson
corr, pval = pearsonr(df['felicidad'].astype(float), df['escala'].astype(float))

print(f"Correlation (Pearson): {corr:.4f}")
print(f"P-value: {pval:.4f}")


Correlation (Pearson): 0.0152
P-value: 0.0537


In [139]:
import pandas as pd
import numpy as np
from scipy.stats import pearsonr

df = latinobarometro.copy()

# Asegurar que la escala es numérica (ignorar errores)
df["escala"] = pd.to_numeric(df["escala"], errors="coerce")

# Asegurar que felicidad es numérica
df["felicidad"] = pd.to_numeric(df["felicidad"], errors="coerce")

# --- 1) Polarización por ciudad: desviación estándar ---
polar_ciudad = df.groupby("ciudad")["escala"].std().reset_index()
polar_ciudad.columns = ["ciudad", "polarizacion_sd"]


# --- 2) (Opcional) Polarización por extremos ---
def extremos(x):
    x = x.dropna()
    if len(x) == 0:
        return np.nan
    return ((x < 2) | (x > 8)).mean()  # proporción en extremos

polar_ciudad["polarizacion_extremos"] = df.groupby("ciudad")["escala"].apply(extremos).values


# --- 3) Felicidad promedio por ciudad ---
felicidad_ciudad = df.groupby("ciudad")["felicidad"].mean().reset_index()
felicidad_ciudad.columns = ["ciudad", "felicidad_prom"]

# Merge
merged = felicidad_ciudad.merge(polar_ciudad, on="ciudad", how="left").dropna()

# --- 4) Correlación: polarización vs felicidad promedio ---
corr_sd, p_sd = pearsonr(merged["felicidad_prom"], merged["polarizacion_sd"])
corr_ext, p_ext = pearsonr(merged["felicidad_prom"], merged["polarizacion_extremos"])

print("Correlación felicidad promedio ~ polarización (SD):", corr_sd, "p=", p_sd)
print("Correlación felicidad promedio ~ polarización (extremos):", corr_ext, "p=", p_ext)


/tmp/ipython-input-2375488557.py:14: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  polar_ciudad = df.groupby("ciudad")["escala"].std().reset_index()
/tmp/ipython-input-2375488557.py:25: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  polar_ciudad["polarizacion_extremos"] = df.groupby("ciudad")["escala"].apply(extremos).values


Correlación felicidad promedio ~ polarización (SD): 0.21423441451780467 p= 2.381618926176736e-14
Correlación felicidad promedio ~ polarización (extremos): 0.217459784578625 p= 9.498863056580367e-15


/tmp/ipython-input-2375488557.py:29: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  felicidad_ciudad = df.groupby("ciudad")["felicidad"].mean().reset_index()


In [140]:
import pandas as pd
from scipy.stats import pearsonr
import numpy as np

# Filtrar datos válidos
df = latinobarometro[['idenpa','felicidad','escala']].replace("#NA", np.nan).dropna()

# Función para porcentaje de extremos ideológicos
def extremos(x):
    return ((x == 0) | (x == 10)).mean()

# Polarización por país
polar_pais = df.groupby("idenpa", observed=False)["escala"].agg(
    polar_sd = 'std',            # dispersión ideológica
    polar_ext = extremos         # % extremos ideológicos
).reset_index()

# Felicidad promedio por país
felic_pais = df.groupby("idenpa", observed=False)["felicidad"].mean().reset_index(name="felicidad_prom")

# Merge
merged = polar_pais.merge(felic_pais, on="idenpa")

# Calcular correlaciones
corr_sd, p_sd = pearsonr(merged["felicidad_prom"], merged["polar_sd"])
corr_ext, p_ext = pearsonr(merged["felicidad_prom"], merged["polar_ext"])

print("Correlación felicidad país ~ polarización (SD):", corr_sd, "p=", p_sd)
print("Correlación felicidad país ~ polarización (extremos):", corr_ext, "p=", p_ext)
print("\nTabla de países:")
print(merged)



Correlación felicidad país ~ polarización (SD): 0.6117420764906453 p= 0.006977273056358408
Correlación felicidad país ~ polarización (extremos): 0.6192812222890958 p= 0.006131916032067055

Tabla de países:
              idenpa  polar_sd  polar_ext  felicidad_prom
0          Argentina  2.377213   0.128041        2.719590
1            Bolivia  2.619663   0.177312        2.591992
2             Brasil  2.877425   0.204198        2.669847
3              Chile  2.116546   0.068413        2.521106
4           Colombia  2.680660   0.173585        3.266981
5         Costa Rica  2.833996   0.223289        3.232893
6    Rep. Dominicana  3.762789   0.515317        3.253829
7            Ecuador  3.027315   0.287596        2.971460
8        El Salvador  2.764292   0.247104        3.181467
9          Guatemala  3.381924   0.388451        3.309711
10          Honduras  3.573034   0.420526        3.117647
11           MÃ©xico  2.645520   0.189071        3.162842
12         Nicaragua  3.801090   0.51001

In [141]:
!wget https://ourworldindata.org/grapher/gdp-per-capita-worldbank.csv?v=1&csvType=filtered&useColumnShortNames=false&tab=table&overlay=download-data

--2025-11-04 18:53:38--  https://ourworldindata.org/grapher/gdp-per-capita-worldbank.csv?v=1
Resolving ourworldindata.org (ourworldindata.org)... 172.66.164.52, 104.20.21.76, 2606:4700:10::6814:154c, ...
Connecting to ourworldindata.org (ourworldindata.org)|172.66.164.52|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 222370 (217K) [text/csv]
Saving to: ‘gdp-per-capita-worldbank.csv?v=1.3’

gdp-per-capita-worl 100%[===================>] 217.16K  --.-KB/s    in 0.01s   

2025-11-04 18:53:38 (19.3 MB/s) - ‘gdp-per-capita-worldbank.csv?v=1.3’ saved [222370/222370]



In [142]:
gdp=pd.read_csv('/content/gdp-per-capita-worldbank.csv?v=1')

In [143]:
import unicodedata

def normalize(s):
    return (unicodedata.normalize('NFKD', s)
            .encode('ascii', 'ignore')
            .decode('utf-8')
            .upper()
            .strip())

latinobarometro['idenpa_norm'] = latinobarometro['idenpa_clean'].apply(normalize)


In [144]:
mapping = {
    "ARGENTINA": "Argentina",
    "BOLIVIA": "Bolivia",
    "BRASIL": "Brazil",
    "COLOMBIA": "Colombia",
    "COSTA RICA": "Costa Rica",
    "CHILE": "Chile",
    "ECUADOR": "Ecuador",
    "EL SALVADOR": "El Salvador",
    "GUATEMALA": "Guatemala",
    "HONDURAS": "Honduras",
    "MEXICO": "Mexico",
    "NICARAGUA": "Nicaragua",
    "PANAMA": "Panama",
    "PARAGUAY": "Paraguay",
    "PERU": "Peru",
    "REP. DOMINICANA": "Dominican Republic",
    "URUGUAY": "Uruguay",
    "VENEZUELA": "Venezuela"
}


In [145]:
latinobarometro['pais_eng'] = latinobarometro['idenpa_norm'].map(mapping)

In [165]:
import unicodedata
import difflib
import pandas as pd

# --- helper: normalizar nombres ---
def normalize_name(s):
    if pd.isna(s):
        return ""
    s = str(s)
    s = s.strip().lower()
    s = unicodedata.normalize("NFKD", s)
    s = "".join(ch for ch in s if not unicodedata.combining(ch))
    s = " ".join(s.split())
    return s

# --- helper: intentar arreglar mojibake latin1->utf-8 ---
def fix_mojibake(s):
    if pd.isna(s):
        return s
    s = str(s)
    # si no contiene la secuencia típica de mojibake, devolvemos
    if "Ã" not in s and "Â" not in s:
        return s
    try:
        candidate = s.encode("latin1").decode("utf-8")
    except Exception:
        return s
    # elegimos el que tenga más letras/espacios (heurística simple)
    def score(u):
        return sum(1 for c in u if c.isalpha() or c.isspace())
    return candidate if score(candidate) > score(s) else s

# --- aplica reparación y normalización a merged y a gdp_latest.pais (por si acaso) ---
merged_df = merged.copy()
gdp_latest = gdp_latest.copy()

merged_df["idenpa_fix"] = merged_df["idenpa"].apply(fix_mojibake)
merged_df["idenpa_norm"] = merged_df["idenpa_fix"].apply(normalize_name)

gdp_latest["pais_fix"] = gdp_latest["pais"].apply(fix_mojibake)
gdp_latest["pais_norm"] = gdp_latest["pais_fix"].apply(normalize_name)

gdp_name_set = set(gdp_latest["pais_norm"].unique())

# --- mapa manual (agregá/edita si hace falta) ---
mapa_paises = {
    "argentina": "Argentina",
    "bolivia": "Bolivia",
    "brasil": "Brazil",
    "chile": "Chile",
    "colombia": "Colombia",
    "costa rica": "Costa Rica",
    "rep. dominicana": "Dominican Republic",
    "rep dominicana": "Dominican Republic",
    "republica dominicana": "Dominican Republic",
    "ecuador": "Ecuador",
    "el salvador": "El Salvador",
    "guatemala": "Guatemala",
    "honduras": "Honduras",
    "mexico": "Mexico",
    "méxico": "Mexico",
    "nicaragua": "Nicaragua",
    "panama": "Panama",
    "panamá": "Panama",
    "paraguay": "Paraguay",
    "peru": "Peru",
    "perú": "Peru",
    "uruguay": "Uruguay",
    "venezuela": "Venezuela"
}
mapa_paises_norm = { normalize_name(k): v for k,v in mapa_paises.items() }

# --- primer pase: mapa directo ---
merged_df["pais_owid"] = merged_df["idenpa_norm"].map(mapa_paises_norm)

# --- segundo pase: fuzzy matching para los que quedan sin mapear ---
def fuzzy_lookup_to_owid(norm_name, cutoff=0.60):
    if not norm_name:
        return None
    if norm_name in gdp_name_set:
        return gdp_latest.loc[gdp_latest["pais_norm"] == norm_name, "pais_fix"].iat[0]
    matches = difflib.get_close_matches(norm_name, list(gdp_name_set), n=1, cutoff=cutoff)
    if matches:
        matched_norm = matches[0]
        return gdp_latest.loc[gdp_latest["pais_norm"] == matched_norm, "pais_fix"].iat[0]
    return None

mask_na = merged_df["pais_owid"].isna()
for idx in merged_df[mask_na].index:
    norm = merged_df.at[idx, "idenpa_norm"]
    candidate = fuzzy_lookup_to_owid(norm, cutoff=0.60)
    merged_df.at[idx, "pais_owid"] = candidate

# --- mostrar quién quedó sin mapear ---
no_map = merged_df[merged_df["pais_owid"].isna()][["idenpa", "idenpa_fix", "idenpa_norm"]]
print("Quedaron sin mapear (después de reparación y fuzzy):")
print(no_map.to_string(index=False))

# --- para cada no mapeado, mostrar las mejores coincidencias de gdp_latest (n=5) ---
if not no_map.empty:
    print("\nSugerencias desde gdp_latest para cada no mapeado:")
    for idx in no_map.index:
        norm = merged_df.at[idx, "idenpa_norm"]
        print(f"\n{merged_df.at[idx,'idenpa']}  -> normalizado='{norm}'")
        cands = difflib.get_close_matches(norm, list(gdp_name_set), n=5, cutoff=0.4)
        if cands:
            for c in cands:
                display_name = gdp_latest.loc[gdp_latest["pais_norm"] == c, "pais_fix"].iat[0]
                print("   -", display_name, " (norm:", c, ")")
        else:
            print("   - No se encontraron candidatos cercanos (baja cutoff si querés).")

# --- reconvertir a string y merge final ---
merged_df["pais_owid"] = merged_df["pais_owid"].astype("string")
gdp_latest["pais_fix"] = gdp_latest["pais_fix"].astype("string")

merged_final = merged_df.merge(
    gdp_latest[["pais_fix", "gdp_pc"]],
    left_on="pais_owid",
    right_on="pais_fix",
    how="left"
).drop(columns=["pais_fix", "idenpa_fix"])

Quedaron sin mapear (después de reparación y fuzzy):
Empty DataFrame
Columns: [idenpa, idenpa_fix, idenpa_norm]
Index: []

Primeras filas del resultado:
    idenpa  polar_sd  polar_ext  felicidad_prom pais_owid idenpa_norm    gdp_pc
 Argentina  2.377213   0.128041        2.719590 Argentina   argentina 26547.050
   Bolivia  2.619663   0.177312        2.591992   Bolivia     bolivia  9844.276
    Brasil  2.877425   0.204198        2.669847    Brazil      brasil 19647.910
     Chile  2.116546   0.068413        2.521106     Chile       chile 30182.787
  Colombia  2.680660   0.173585        3.266981  Colombia    colombia 18503.670

Filas sin gdp_pc tras merge (si aparecen, hay que afinar el mapa):
    idenpa pais_owid
 Venezuela Venezuela


In [167]:
import statsmodels.api as sm

# Subset solo columnas necesarias y drop de filas sin datos
df = merged_final[['felicidad_prom', 'polar_ext', 'gdp_pc']].dropna()

# Definir X y Y
X = df[['polar_ext', 'gdp_pc']]
X = sm.add_constant(X)   # agregar constante
y = df['felicidad_prom']

# Ajustar modelo OLS con errores robustos HC3
model = sm.OLS(y, X).fit(cov_type='HC3')
print(model.summary())


                            OLS Regression Results                            
Dep. Variable:         felicidad_prom   R-squared:                       0.419
Model:                            OLS   Adj. R-squared:                  0.336
Method:                 Least Squares   F-statistic:                     5.593
Date:                Tue, 04 Nov 2025   Prob (F-statistic):             0.0164
Time:                        19:29:18   Log-Likelihood:                 3.7467
No. Observations:                  17   AIC:                            -1.493
Df Residuals:                      14   BIC:                             1.006
Df Model:                           2                                         
Covariance Type:                  HC3                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          2.5616      0.179     14.332      0.0